In [8]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

rental_incomeDF = pd.read_csv('output_data/rental(total)_income_merged.csv')
rental_incomeDF


#Data Formatting 
rental_incomeDF['2015 index'] = (rental_incomeDF['2015 Yearly Rent']/rental_incomeDF['2015 Income']*100).map("{:.1f}".format)
rental_incomeDF['2016 index'] = (rental_incomeDF['2016 Yearly Rent']/rental_incomeDF['2016 Income']*100).map("{:.1f}".format)
rental_incomeDF['2017 index'] = (rental_incomeDF['2017 Yearly Rent']/rental_incomeDF['2017 Income']*100).map("{:.1f}".format)

rental_incomeDF =rental_incomeDF.reset_index(drop=True)
rental_incomeDF.head()


,State,2015 Yearly Rent,2016 Yearly Rent,2017 Yearly Rent,'15-'16 Rent Diff.,'16-'17 Rent Diff.,2015 Income,2016 Income,2017 Income,'15-'16 Income Diff.,'16-'17 Income Diff.,2015 index,2016 index,2017 index
0,Alabama,11400.0,11700.0,11970.0,300.0,270.0,46053,48237,51113,2184,2876,24.8,24.3,23.4
1,Alaska,20190.0,19800.0,19170.0,-390.0,-630.0,77717,77351,72231,-366,-5120,26.0,25.6,26.5
2,Arizona,14700.0,15000.0,16200.0,300.0,1200.0,54060,58328,61125,4268,2797,27.2,25.7,26.5
3,Arkansas,12000.0,12000.0,12600.0,0.0,600.0,44282,46894,48829,2612,1935,27.1,25.6,25.8
4,California,27510.0,28740.0,31500.0,1230.0,2760.0,65843,68070,69759,2227,1689,41.8,42.2,45.2


In [56]:
#input annual income 
income = (input("What is your yearly[2017] income? " ))
input_rent = int(int(income)*0.3)
print(f"You can afford a rent up to {input_rent} annually or {int(input_rent/12)} monthly.")

#create new DF with relevant columns
affordable_rentDF_input = rental_incomeDF[['State','2017 Yearly Rent',]]

#calculate rent/income index
affordable_rentDF_input['index'] = (affordable_rentDF_input['2017 Yearly Rent']/int(income)*100).map("{:.1f}".format)

#sort states by rent/income index in ascending order
affordable_rentDF_input = affordable_rentDF_input.sort_values('index').reset_index(drop=True)

#Filter states with rent/income index <= 30
affordable = affordable_rentDF_input.loc[affordable_rentDF_input['index'].astype(float)<=30].count()['index']
print(f'\nIn 2017 with an income of {income} you could afford rent comfortably in {affordable} states.')

#format yearly rent column 
affordable_rentDF_input['2017 Yearly Rent'] = affordable_rentDF_input['2017 Yearly Rent'].map("${:,.0f}".format)

#Filter states with rent/income index > 30
not_affordable = len(affordable_rentDF_input.tail(48-affordable))
not_affordable_states = affordable_rentDF_input.tail(48-affordable)
print(f'You cannot comfortably afford rent in the following {not_affordable} states.')
not_affordable_states

What is your yearly[2017] income? 100000
You can afford a rent up to 30000 annually or 2500 monthly.

In 2017 with an income of 100000 you could afford rent comfortably in 45 states.
You cannot comfortably afford rent in the following 3 states.


,State,2017 Yearly Rent,index
45,District of Columbia,"$31,200",31.2
46,California,"$31,500",31.5
47,New York,"$38,700",38.7


In [40]:
#create heat-map to show which states the user can afford rent given the income input (index <= 30)
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import mpl
from bokeh.plotting import figure
import geopandas as gpd
from bokeh.plotting import figure, output_file, show

output_file("Affordable Rent 2017 - Based on Income Input.html")




contiguous_usa = gpd.read_file('cb_2018_us_state_20m/cb_2018_us_state_20m.shp')
contiguous_usa.columns = ['STATEFP', 'STATENS', 'AFFGEOID', 'GEOID', 'STUSPS', 'State', 'LSAD',
       'ALAND', 'AWATER', 'geometry']
contiguous_usa.head()

index_input = pd.merge(contiguous_usa,affordable_rentDF_input , on="State", suffixes=("_shape", ""))
index_input = index_input.loc[~index_input['State'].isin(['Nebraska','Hawaii', 'Alaska'])]
index_input

geosource_index_input = GeoJSONDataSource(geojson = index_input.to_json())

## Plot 2017 rent/income index 
# Define color palettes
palette = mpl['Plasma'][6]
palette = palette[::-1] # reverse order of colors so higher values have darker colors
# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 20, high = 80) #make sure you set the low and high, if you forget it won't map
# Define custom tick labels for color bar.
#tick_labels = {'0': '30', '30':'30+'}

# Create color bar.
color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 8,
                     width = 300, height = 20,
                     border_line_color = None,
                     location = (0,0), 
                     orientation = 'horizontal')

# Create figure object.
p_input = figure(title = f'Rental Price/Median Income({income}) Index - Affordable Rent < 30', 
           plot_height = 600, plot_width = 950, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p_input.xgrid.grid_line_color = None
p_input.ygrid.grid_line_color = None
# Add patch renderer to figure.
states = p_input.patches('xs','ys', source = geosource_index_input, #your geojson file
                   fill_color = {'field' :'index',
                                 'transform' : color_mapper},
                   line_color = 'gray', 
                   line_width = 0.25, 
                   fill_alpha = 1)

# Create hover tool
p_input.add_tools(HoverTool(renderers = [states],
                      tooltips = [('State','@State'),
                               ('Income Change','@index{int}')])) #if you add {int} then it writes as integer, otherwise scientific notation
                                                                #the @ shows the column name, so here I am saying "in hover box show "Median Rent: [value in 'Value' column]{in integer format}
# Specify layout
p_input.add_layout(color_bar, 'below')

show(p_input)

In [11]:
#Create Affordable Rent (rent/income index <= 30) for median rent & median hh income -- 2017 ONLY
affordable_rentDF2017 = rental_incomeDF.loc[rental_incomeDF['2017 index'].astype(float)<=30.0].sort_values('2017 index')
affordable_rentDF2017 = affordable_rentDF2017[['State','2017 Yearly Rent', '2017 Income', '2017 index']].reset_index(drop=True)
affordable_rentDF2017.head()

#create heat-map of rent/income index -- 2017 ONLY
from bokeh.plotting import figure, output_file, show

output_file("Affordable Rent 2017.html")

index2017 = pd.merge(contiguous_usa, rental_incomeDF, on="State", suffixes=("_shape", ""))
index2017 = index2017.loc[~index2017['State'].isin(['Nebraska','Hawaii', 'Alaska'])]
index2017

geosource_index2017 = GeoJSONDataSource(geojson = index2017.to_json())

# Define color palettes
palette = mpl['Plasma'][6]
palette = palette[::-1] # reverse order of colors so higher values have darker colors
# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 20, high = 80) #make sure you set the low and high, if you forget it won't map


# Create color bar.
color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 8,
                     width = 300, height = 20,
                     border_line_color = None,
                     location = (0,0), 
                     orientation = 'horizontal')

# Create figure object.
p_2017index = figure(title = '2017 Rental Price/Median Income Index (Affordable Rent =< 30 Index)', 
           plot_height = 600, plot_width = 950, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p_2017index.xgrid.grid_line_color = None
p_2017index.ygrid.grid_line_color = None
# Add patch renderer to figure.
states = p_2017index.patches('xs','ys', source = geosource_index2017, #your geojson file
                   fill_color = {'field' :'2017 index',
                                 'transform' : color_mapper},
                   line_color = 'gray', 
                   line_width = 0.25, 
                   fill_alpha = 1)

# Create hover tool
p_2017index.add_tools(HoverTool(renderers = [states],
                      tooltips = [('State','@State'),
                               ('Income Change','@{2017 index}{int}')])) #if you add {int} then it writes as integer, otherwise scientific notation
                                                                #the @ shows the column name, so here I am saying "in hover box show "Median Rent: [value in 'Value' column]{in integer format}
# Specify layout
p_2017index.add_layout(color_bar, 'below')

show(p_2017index)

In [12]:
#Create Affordable Rent (rent/income index <= 30) for median rent & median hh income -- 2016 ONLY
affordable_rentDF2016 = rental_incomeDF.loc[rental_incomeDF['2016 index'].astype(float)<=30.0].sort_values('2016 index')
affordable_rentDF2016 = affordable_rentDF2016[['State','2016 Yearly Rent', '2016 Income', '2016 index']].reset_index(drop=True)
affordable_rentDF2016.head()

#create heat-map of rent/income index -- 2017 ONLY
from bokeh.plotting import figure, output_file, show

output_file("Affordable Rent 2016.html")

index2016 = pd.merge(contiguous_usa, rental_incomeDF, on="State", suffixes=("_shape", ""))
index2016 = index2016.loc[~index2016['State'].isin(['Nebraska','Hawaii', 'Alaska'])]
index2016

geosource_index2016 = GeoJSONDataSource(geojson = index2016.to_json())


# Define color palettes
palette = mpl['Plasma'][6]
palette = palette[::-1] # reverse order of colors so higher values have darker colors
# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 20, high = 80) #make sure you set the low and high, if you forget it won't map


# Create color bar.
color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 8,
                     width = 300, height = 20,
                     border_line_color = None,
                     location = (0,0), 
                     orientation = 'horizontal')

# Create figure object.
p_2016index = figure(title = '2016 Rental Price/Median Income Index (Affordable Rent =< 30 Index)', 
           plot_height = 600, plot_width = 950, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p_2016index.xgrid.grid_line_color = None
p_2016index.ygrid.grid_line_color = None
# Add patch renderer to figure.
states = p_2016index.patches('xs','ys', source = geosource_index2016, #your geojson file
                   fill_color = {'field' :'2016 index',
                                 'transform' : color_mapper},
                   line_color = 'gray', 
                   line_width = 0.25, 
                   fill_alpha = 1)

# Create hover tool
p_2016index.add_tools(HoverTool(renderers = [states],
                      tooltips = [('State','@State'),
                               ('Income Change','@{2016 index}{int}')])) #if you add {int} then it writes as integer, otherwise scientific notation
                                                                #the @ shows the column name, so here I am saying "in hover box show "Median Rent: [value in 'Value' column]{in integer format}
# Specify layout
p_2016index.add_layout(color_bar, 'below')

show(p_2016index)

In [13]:
#Create Affordable Rent (rent/income index <= 30) for median rent & median hh income -- 2015 ONLY
affordable_rentDF2015 = rental_incomeDF.loc[rental_incomeDF['2015 index'].astype(float)<=30.0].sort_values('2015 index')
affordable_rentDF2015 = affordable_rentDF2015[['State','2015 Yearly Rent', '2015 Income', '2015 index']].reset_index(drop=True)
affordable_rentDF2015.head()

#create heat-map of rent/income index -- 2017 ONLY
from bokeh.plotting import figure, output_file, show

output_file("Affordable Rent 2015.html")

index2015 = pd.merge(contiguous_usa, rental_incomeDF, on="State", suffixes=("_shape", ""))
index2015 = index2015.loc[~index2015['State'].isin(['Nebraska','Hawaii', 'Alaska'])]
index2015

geosource_index2015 = GeoJSONDataSource(geojson = index2015.to_json())


# Define color palettes
palette = mpl['Plasma'][6]
palette = palette[::-1] # reverse order of colors so higher values have darker colors
# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 20, high = 80) #make sure you set the low and high, if you forget it won't map


# Create color bar.
color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 8,
                     width = 300, height = 20,
                     border_line_color = None,
                     location = (0,0), 
                     orientation = 'horizontal')

# Create figure object.
p_2015index = figure(title = '2015 Rental Price/Median Income Index (Affordable Rent =< 30 Index)', 
           plot_height = 600, plot_width = 950, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p_2015index.xgrid.grid_line_color = None
p_2015index.ygrid.grid_line_color = None
# Add patch renderer to figure.
states = p_2015index.patches('xs','ys', source = geosource_index2015, #your geojson file
                   fill_color = {'field' :'2015 index',
                                 'transform' : color_mapper},
                   line_color = 'gray', 
                   line_width = 0.25, 
                   fill_alpha = 1)

# Create hover tool
p_2015index.add_tools(HoverTool(renderers = [states],
                      tooltips = [('State','@State'),
                               ('Income Change','@{2015 index}{int}')])) #if you add {int} then it writes as integer, otherwise scientific notation
                                                                #the @ shows the column name, so here I am saying "in hover box show "Median Rent: [value in 'Value' column]{in integer format}
# Specify layout
p_2015index.add_layout(color_bar, 'below')

show(p_2015index)

In [14]:
count_2015 = rental_incomeDF.loc[rental_incomeDF['2015 index'].astype(float)>30].count()['2015 index']
print(f'In 2015 there were {count_2015} states whose rent/income index was > 30.')

count_2016 = rental_incomeDF.loc[rental_incomeDF['2016 index'].astype(float)>30].count()['2016 index']
print(f'In 2016 there were {count_2016} states whose rent/income index was > 30.')

count_2017 = rental_incomeDF.loc[rental_incomeDF['2017 index'].astype(float)>30].count()['2017 index']
print(f'In 2017 there were {count_2017} states whose rent/income index was > 30.')

In 2015 there were 12 states whose rent/income index was > 30.
In 2016 there were 12 states whose rent/income index was > 30.
In 2017 there were 14 states whose rent/income index was > 30.
